In [13]:
import pandas as pd
import json
import re


In [8]:

output_zeroshot = r"batch output/BatchOpenAI_GPT4o_0_batch_67f80fa9d50c81908a88bae5f9a7f3bd_output.jsonl"
output_fewshot = r"batch output\BatchOpenAI_GPT4o_1_batch_67f80fbbbc608190b9bae2e94287ba6a_output.jsonl"
output_cot =r"batch output\BatchOpenAI_GPT4o_2_batch_67f80fc765a88190b7a38ba48ed271d4_output.jsonl"

outoput =[output_zeroshot, output_fewshot, output_cot]


In [9]:
def extract_contents_from_jsonl(path):
    contents = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            try:
                content = data["response"]["body"]["choices"][0]["message"]["content"]
                contents.append(content)
            except KeyError:
                contents.append("")
    return contents

In [11]:
def separar_analisis_y_polaridad(respuestas):
    analisis = []
    polaridades = []

    for r in respuestas:
        match = re.search(r"(.*)4\.\s*Polaridad final:\s*(POS|NEG|NEU)", r, re.DOTALL)
        if match:
            analisis.append(match.group(1).strip())
            polaridades.append(match.group(2))
        else:
            analisis.append(r.strip())
            polaridades.append("NO_MATCH")

    return analisis, polaridades

In [16]:
zeroShot = extract_contents_from_jsonl(output_zeroshot)
fewShot = extract_contents_from_jsonl(output_fewshot)
changeOfThought = extract_contents_from_jsonl(output_cot)

analisis_changeOfThought, polaridad_changeOfThought = separar_analisis_y_polaridad(changeOfThought)

In [20]:
df_zeroShot = pd.DataFrame(zeroShot, columns=["Polarity_GPT-4o-mini_ZeroShot"])
df_fewShot = pd.DataFrame(fewShot, columns=["Polarity_GPT-4o-mini_FewShot"])
df_pol_changeOfThought = pd.DataFrame(polaridad_changeOfThought, columns=["Polarity_GPT-4o-mini_Chain_Of_Thought"])
df_raw_changeOfThought = pd.DataFrame(analisis_changeOfThought, columns=["Raw_GPT-4o-mini_Chain_Of_Thought"])

In [ ]:
df = pd.read_excel('../Spain_train_set_2019.xlsx')

df["Polarity_GPT-4o-mini_ZeroShot"] = df_zeroShot
df["Polarity_GPT-4o-mini_FewShot"] = df_fewShot
df["Polarity_GPT-4o-mini_Chain_Of_Thought"] = df_pol_changeOfThought
df["Raw_GPT-4o-mini_Chain_Of_Thought"] = df_raw_changeOfThought
df



,ID,Content,Lang,Polarity,Polarity_GPT-4o-mini_ZeroShot,Polarity_GPT-4o-mini_FewShot,Polarity_GPT-4o-mini_Chain_Of_Thought,Raw_GPT-4o-mini_Chain_Of_Thought
0,768213567418036224,@myendlesshazza a. que puto mal escribo b. me ...,es,NEG,NEG,NEG,POS,"1. Identificación de palabras clave: ""puto mal..."
1,768221670255493120,Quiero mogollón a @AlbaBenito99 pero sobretodo...,es,POS,POS,POS,POS,"1. Identificación de palabras clave: ""Quiero m..."
2,768221021300264964,Vale he visto la tia bebiendose su regla y me ...,es,NEG,NEG,NEG,NEG,"1. Identificación de palabras clave: ""bebiendo..."
3,768220253730009091,@Yulian_Poe @guillermoterry1 Ah. mucho más por...,es,POS,NEU,NEU,NEU,"1. Identificación de palabras clave: ""mucho má..."
4,768231229439311872,@toNi_end seria mejor que dejasen de emitir es...,es,NEG,NEG,NEG,NEG,"1. Identificación de palabras clave: ""mejor"", ..."
...,...,...,...,...,...,...,...,...
963,814846333601320960,Para mí mi mejor año fue el 2015,es,POS,POS,NEU,POS,"1. Identificación de palabras clave: ""mejor añ..."
964,813731371076243461,Hoy va a ser un dia muy largo...,es,NEG,NEG,NEG,NEG,"1. Identificación de palabras clave: ""día muy ..."
965,818399956792905728,11:11 que no me encuentre mal,es,NEU,NEG,NEG,NEU,"1. Identificación de palabras clave: ""no me en..."
966,815715581878009858,Es que son muy bonitas jo,es,POS,POS,POS,POS,"1. Identificación de palabras clave: ""muy boni..."


In [22]:
df.to_excel('Clasificación_GPT-4o-mini.xlsx', index=False)